In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

import numpy as np
import pandas as pd
import math
from scipy.optimize import fsolve
from scipy.optimize import fmin
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from os import listdir
%matplotlib inline

import base64
import datetime
import io

from textwrap import dedent as d
from scipy import stats
from decimal import Decimal

import plotly.graph_objs as go

In [2]:
app = dash.Dash()

In [3]:
# Define graph layout for linear regime calculations

abs_layout = go.Layout(
    xaxis={'title': 'Vg (V)'},
    yaxis={'title': '|Id| (A)'},
    height=350,
    margin=go.Margin(
        l=35,
        r=5,
        b=40,
        t=5),
    dragmode='select',
    legend=dict(
        borderwidth=2,
        xanchor='right')
)

C:\Users\Joshua Wen-Kai Huang\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [4]:
# Format page, reference utilities, markdown explanations

app.layout = html.Div([
    html.H1(html.B('Transfer Curve Analysis')),
    
    html.Div([
        html.Div([dcc.Markdown(d('''
            - Ensure that the file is a .txt format file that only has one table within it
            - Gate Voltage should be in a column titled 'Vg(V)', Absolute Drain Current titled '-Id(A)', and Sqrt Drain Current titled 'Sqrt(Id)'
            - If Vd > Vg-Vt, (e.g. Vd = -100 V), then you are calculating saturated mobility
            - If Vd << Vg, (e.g. Vd = -10 V), then you are calculating linear mobility
            - Threshold voltage (Vt) is calculated from transfer curve in the Saturated Regime''')),
            html.Div([html.B(html.I(
            'Linear mobility is the preferred value for publications'
            ))], 
            style = {
            'fontSize': 20,
            }),
            dcc.Upload(id = 'upload-data',
                       children = html.Button('Upload File'))
                 ],
        style = {
            'maxWidth': 900,
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '20px',
            'borderColor': 'rgb(160, 160, 160)',
            'padding': 20,
            'margin': '0px auto'
        })
    ]),
    
    html.Hr(),
   
    html.Div([
        html.Div([
            html.H3('Vg Selection'),    
            html.Div('Select the range of Vg, where the curve is linear, using the Box Select Tool below:'),
            dcc.Graph(
                id='transfer-curve',
                figure=go.Figure(),
                selectedData={'points': [], 'range': None}
            )
        ], style={'width': '50%', 'display': 'inline-block'})])
    

    
])

In [5]:
# Template for compiling and designing webpage

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

In [6]:
# Accesses uploaded data and defines 3 dataframes, corresponding to Vg, |Id|, and sqrt(Id)

def dataframe(contents):
    
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    Ldata = pd.read_table(io.StringIO(decoded.decode('utf-8')))
        
    idx_end = Ldata[Ldata.iloc[:,0] == 'Jsc:'].index[0]
    Ldata = Ldata.iloc[:idx_end-1,:]
    Ldata.iloc[:,0] = pd.to_numeric(Ldata.iloc[:,0])
    Ldata.iloc[:,0]
    Ldata = np.array(Ldata)


    Ldata = np.insert(Ldata, 2, -Ldata[:,1], axis=1)


    #Sort data by ascending V, so less confusing
    Ldata = Ldata[Ldata[:,0].argsort()]

    return Ldata

#     df_Vg = df['Vg(V)'] 
#     df_Id = df['Id(A)']
#     df_sqrtId = df['Sqrt(Id)']
    
#     Vg = df_Vg.values
#     Id = df_Id.values
#     sqrtId = df_sqrtId.values
    
#     absId = np.zeros(len(Id))
    
#     for i in range(len(Id)):
#         absId[i] = abs(Id[i])
    
#     return Vg, absId, sqrtId

In [7]:
# Displays uploaded data

@app.callback(Output('transfer-curve', 'figure'),
             [Input('upload-data', 'contents')])
def display_uploaded_data(contents):
    Ldata = dataframe(contents)
    
    a_layout = abs_layout
    
    return go.Figure(
        data=[
            go.Scatter(
                x=Ldata[:,0],
                y=Ldata[:,2],
                mode='lines+markers'
            )
        ],
        layout=a_layout)

In [ ]:
# Opens browser page to host dashboard

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
C:\Users\Joshua Wen-Kai Huang\Anaconda3\lib\site-packages\dash\resources.py:44: UserWarning:

A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available

127.0.0.1 - - [06/Aug/2019 12:45:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2019 12:45:37] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563309670 HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2019 12:45:37] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563309670 HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2019 12:45:37] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563309670 HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2019 12:45:37] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563309546 HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2019 12:45:37] "GET /_dash-component-suites/dash_table/bundle.js?v=4.0.2&m=1563309547 HTTP/1.1" 200 -
127.0.0.1 - - [06/Au